In [ ]:
!mkdir -p ../../audio_set

In [ ]:
# Download data from http://storage.googleapis.com/us_audioset/youtube_corpus/v1/csv/unbalanced_train_segments.csv 
# Put it in ../../data

In [4]:
import io
import pandas as pd
from collections import deque

In [50]:
ROOT_DIR="../../data/audio_set"
RAW_FILE="{}/{}".format(ROOT_DIR, "unbalanced_train_segments.csv")
CLEANED_FILE="{}/{}".format(ROOT_DIR, "cleaned_train_segments.csv")

PANDAS_IN_FILE="{}/{}".format(ROOT_DIR, "cough_in_train_segments.csv")

In [44]:
# Remove the first 3 lines of file
def efficient_dropfirst(f, nf, dropfirst=1, buffersize=3):
    f.seek(0)
    buffer = deque()
    tail_pos = 0
    # these next two loops assume the file has many thousands of
    # lines so we can safely drop and buffer the first few...
    for _ in range(dropfirst):
        f.readline()
    for _ in range(buffersize):
        buffer.append(f.readline())
    line = f.readline()
    while line:
        buffer.append(line)
        head_pos = f.tell()
        f.seek(tail_pos)
        tail_pos += nf.write(buffer.popleft())
        f.seek(head_pos)
        line = f.readline()
    f.seek(tail_pos)
    # finally, clear out the buffer:
    while buffer:
        line = buffer.popleft()
        nf.write(buffer.popleft())
    nf.truncate()

In [37]:
!head  $RAW_FILE

# Segments csv created Sun Mar  5 10:56:58 2017
# num_ytids=2041789, num_segs=2041789, num_unique_labels=527, num_positive_labels=4020212
# YTID, start_seconds, end_seconds, positive_labels
---1_cCGK4M, 0.000, 10.000, "/m/01g50p,/m/0284vy3,/m/06d_3,/m/07jdr,/m/07rwm0c"
---2_BBVHAA, 30.000, 40.000, "/m/09x0r"
---B_v8ZoBY, 30.000, 40.000, "/m/04rlf"
---EDNidJUA, 30.000, 40.000, "/m/02qldy,/m/02zsn,/m/05zppz,/m/09x0r"
---N4cFAE1A, 21.000, 31.000, "/m/04rlf,/m/09x0r"
---fcVQUf3E, 30.000, 40.000, "/m/019jd,/m/07yv9"
---g9OGAhwc, 30.000, 40.000, "/m/04rlf,/m/0c1dj"


In [46]:
with open(RAW_FILE) as f:
    with open(CLEANED_FILE, "w") as nf:
        efficient_dropfirst(f, nf, 3, 10)

In [40]:
!head -n 2 $CLEANED_FILE

---1_cCGK4M, 0.000, 10.000, "/m/01g50p,/m/0284vy3,/m/06d_3,/m/07jdr,/m/07rwm0c"
---2_BBVHAA, 30.000, 40.000, "/m/09x0r"


In [60]:
!rm $PANDAS_IN_FILE

In [70]:
#clean up for pandas

with open(PANDAS_IN_FILE, "w+") as pin:
    pin.write("""youtubeid,start,end,tags\n""")
    with open(CLEANED_FILE, "r") as cf:
        new_lines = []
        for line in cf:
            cols = line.split(",")
            new_line = ""
            
            new_line = ",".join([cols[0], cols[1], cols[2], " ".join(cols[3:])])
            
            new_lines.append(new_line)
            
        pin.write("\n".join(new_lines))
            
            
            
            

In [71]:
!head -n 2 $PANDAS_IN_FILE

youtubeid,start,end,tags
---1_cCGK4M, 0.000, 10.000, "/m/01g50p /m/0284vy3 /m/06d_3 /m/07jdr /m/07rwm0c"


In [72]:
pd.read_csv(PANDAS_IN_FILE)

,youtubeid,start,end,tags
0,---1_cCGK4M,0.0,10.0,"""/m/01g50p /m/0284vy3 /m/06d_3 /m/07jdr /m/07..."
1,---2_BBVHAA,30.0,40.0,"""/m/09x0r"""
2,---B_v8ZoBY,30.0,40.0,"""/m/04rlf"""
3,---EDNidJUA,30.0,40.0,"""/m/02qldy /m/02zsn /m/05zppz /m/09x0r"""
4,---N4cFAE1A,21.0,31.0,"""/m/04rlf /m/09x0r"""
...,...,...,...,...
2041779,zzyiWF0F9sg,20.0,30.0,"""/m/01j4z9 /m/0_ksk"""
2041780,zzyr6l8MetA,60.0,70.0,"""/m/04rlf /m/05148p4"""
2041781,zzyyleHsxfk,20.0,30.0,"""/m/05tny_ /m/068hy /m/0bt9lr /m/0jbk"""
2041782,zzz3PZXRQ_8,30.0,40.0,"""/m/030rvx /m/09x0r"""


In [0]:
# NEXT STEPS 
# Get cough videos and download and process to PDM using FFMPEG